# Neural Distinguisher of Reduced Round Speck 32/64

## Speck 32/64

Speck 32/64의 구현이 맞는지 테스트벡터를 우선 확인해보겠습니다.

In [ ]:
from speck import Speck

cipher = Speck()
cipher.check_testvector()

## Neural Distinguisher

Speck32/64의 5, 6, 7라운드 distinguisher의 모델을 생성하고 학습시켜보겠습니다. 

사용한 모델은 다음과 같습니다.자세한 코드는 train.py를 참고하면 됩니다.

- 은닉계층이 2개인 DNN
- Residual Block이 하나인 ResNet
- Residual Block이 10개인 ResNet

시간관계상 epoch를 1로 설정했습니다. 

매 epoch를 진행하면서 validation accuracy 가 가장 높은 상태를 .h5 확장자 파일로 저장합니다.

In [ ]:
import train

train.train_all(epochs=1)

epoch이 1이기 때문에 성능이 논문에 적힌 것보다 좋게 나오지 않습니다.

제가 epoch을 200으로 하여 학습시킨 데이터를 trained_models 폴더에 옮겨두었습니다.

해당 모델을 불러들여서 테스트 하는 코드가 evaluate.py에 구현되어 있습니다.

In [ ]:
import evaluate

In [ ]:
#DNN 결과
evaluate.evaluate_dnn()

In [ ]:
#ResNet (block=1) 결과
evaluate.evaluate_resnet(resnet_depth=1)

In [ ]:
#ResNet (block=10) 결과
evaluate.evaluate_resnet(resnet_depth=10)

## Key Recovery Attack

5라운드 distinguisher를 사용하여 6라운드 Speck 32/64의 키를 복구해보겠습니다.

해당 코드는 key_recovery.py 에 구현되어 있습니다.

In [ ]:
import key_recovery

key_recovery.attack(num_test=5, target_round=6)